<a href="https://colab.research.google.com/github/JayThibs/Weak-Supervised-Learning-Case-Study/blob/main/text_classifier/notebooks/03_dbpedia_14_snorkel_dataset_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel --quiet
!pip install datasets --quiet
!pip install spacy --quiet

     |████████████████████████████████| 153kB 10.3MB/s 
     |████████████████████████████████| 3.8MB 8.7MB/s 
     |████████████████████████████████| 1.8MB 63.4MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 225kB 12.6MB/s 
     |████████████████████████████████| 245kB 26.7MB/s 
     |████████████████████████████████| 112kB 32.6MB/s 


In [2]:
import pandas as pd
from snorkel.labeling import labeling_function
from datasets import load_dataset
import re

In [3]:
dbpedia_dataset = load_dataset('dbpedia_14')

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


In [4]:
# dbpedia_dataset = dbpedia_dataset.rename_column("label", "labels") # snorkel uses "label"

In [5]:
def merge_title_with_content(example):
    example["content"] = example["title"] + " " + example["content"]
    return example

In [6]:
dbpedia_dataset = dbpedia_dataset.map(merge_title_with_content, num_proc=10)

In [7]:
dbpedia_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 70000
    })
})

In [68]:
data = {'labels': -1, # replacing with -1 because snorkel needs this for training set, for keyword creation use: dbpedia_dataset['train']['labels']
        'text': dbpedia_dataset['train']['content']} # need to replace "content" with "text" for snorkel to work

dbpedia_train_df = pd.DataFrame(data)
dbpedia_train_df['text'] = dbpedia_train_df['text'].str.lower()
dbpedia_train_df

,labels,text
0,-1,e. d. abbott ltd abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929. a major part of their output was under sub-contract to motor vehicle manufacturers. their business closed in 1972.
1,-1,schwan-stabilo schwan-stabilo is a german maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. it is the world's largest manufacturer of highlighter pens stabilo boss.
2,-1,q-workshop q-workshop is a polish company located in poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). they also run an online retail store and maintainan active forum community.q-workshop was established in 2001 by patryk strzelewicz – a student from poznań. initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished.
3,-1,marvell software solutions israel marvell software solutions israel known as radlan computer communications limited before 2007 is a wholly owned subsidiary of marvell technology group that specializes in local area network (lan) technologies.
4,-1,bergan mercy medical center bergan mercy medical center is a hospital located in omaha nebraska. it is part of the alegent health system.
...,...,...
559995,-1,barking in essex barking in essex is a black comedy play directed by harry burton. it is based on the 2005 script by clive exton and made its world premiere at the wyndham's theatre in september 2013.
559996,-1,science & spirit science & spirit is a discontinued american bimonthly magazine that covered scientific stories with an eye toward their spiritual implications. it was launched by the john templeton foundation in 1989 as a newsletter converted to a glossy magazine in 1998 then repositioned for a general readership in 2001. in 2003 it was purchased by heldref publications though the john templeton foundation continued to provide editorial support.
559997,-1,the blithedale romance the blithedale romance (1852) is nathaniel hawthorne's third major romance. in hawthorne (1879) henry james called it the lightest the brightest the liveliest of hawthorne's unhumorous fictions.
559998,-1,razadarit ayedawbon razadarit ayedawbon (burmese: ရာဇာဓိရာဇ် အရေးတော်ပုံ) is a burmese chronicle covering the history of ramanya from 1287 to 1421. the chronicle consists of accounts of court intrigues rebellions diplomatic missions wars etc. about half of the chronicle is devoted to the reign of king razadarit (r.


In [69]:
data = {'labels': dbpedia_dataset['test']['labels'],
        'text': dbpedia_dataset['test']['content']} # need to replace "content" with "text" for snorkel to work

dbpedia_test_df = pd.DataFrame(data)
del data
dbpedia_test_df['text'] = dbpedia_test_df['text'].str.lower()
dbpedia_test_df

,labels,text
0,0,ty ku ty ku /taɪkuː/ is an american alcoholic beverage company that specializes in sake and other spirits. the privately-held company was founded in 2004 and is headquartered in new york city new york. while based in new york ty ku's beverages are made in japan through a joint venture with two sake breweries. since 2011 ty ku's growth has extended its products into all 50 states.
1,0,odd lot entertainment oddlot entertainment founded in 2001 by longtime producers gigi pritzker and deborah del prete (the wedding planner) is a film production and financing company based in culver city california.oddlot produced the film version of orson scott card's sci-fi novel ender's game. a film version of this novel had been in the works in one form or another for more than a decade by the time of its release.
2,0,henkel henkel ag & company kgaa operates worldwide with leading brands and technologies in three business areas: laundry & home care beauty care and adhesive technologies. henkel is the name behind some of america’s favorite brands.
3,0,goat store the goat store (games of all type store) llc is one of the largest retro gaming online stores and an independent video game publishing label. additionally they are one of the primary sponsors for midwest gaming classic.
4,0,ragwing aircraft designs ragwing aircraft designs (also called the ragwing aeroplane company and ragwing aviation) was an american aircraft design and manufacturing company based in belton south carolina.
...,...,...
69995,13,energy victory energy victory: winning the war on terror by breaking free of oil is a 2007 book by robert zubrin. zubrin's central argument is that the decisive front in the war on terror is america's struggle for energy independence. he outlines the manner in which radical islam has been financed by oil revenues the technological feasibility of ethanol-fueled vehicles as well as the economic and agricultural imperatives for ethanol production and the environmental implications of his plan.
69996,13,bestiario bestiario is a book of 8 short stories written by julio cortázar.
69997,13,wuthering heights wuthering heights is a novel by emily brontë written between october 1845 and june 1846 and published in 1847 under the pseudonym ellis bell. it was her first and only published novel: she died the following year at age 30. the decision to publish came after the success of her sister charlotte's novel jane eyre.
69998,13,l'indépendant l'indépendant is a newspaper published in luxembourg from 1945.


## A gentle introduction to LFs

Labeling functions (LFs) help users encode domain knowledge and other supervision sources programmatically.

LFs are heuristics that take as input a data point and either assign a label to it (in this case, HAM or SPAM) or abstain (don’t assign any label). Labeling functions can be noisy: they don’t have perfect accuracy and don’t have to label every data point. Moreover, different labeling functions can overlap (label the same data point) and even conflict (assign different labels to the same data point). This is expected, and we demonstrate how we deal with this later.

Because their only requirement is that they map a data point a label (or abstain), they can wrap a wide variety of forms of supervision. Examples include, but are not limited to:

    Keyword searches: looking for specific words in a sentence
    Pattern matching: looking for specific syntactical patterns
    Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
    Distant supervision: using external knowledge base
    Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

### a) Exploring the training set for initial ideas

We’ll start by looking at 20 random data points from the train set to generate some ideas for LFs.


In [88]:
ABSTAIN = -1
Company = 0
EducationalInstitution = 1
Artist = 2
Athlete = 3
OfficeHolder = 4
MeanOfTransportation = 5
Building = 6
NaturalPlace = 7
Village = 8
Animal = 9
Plant = 10
Album = 11
Film = 12
WrittenWork = 13

In [54]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

In [56]:
# # for keyword creation
# df_view = dbpedia_train_df[dbpedia_train_df['labels'] == 7].sample(100, random_state=42)
# df_view

In [57]:
# Showing the most common words to make the keyword creation process quicker
# We are choosing some of words in this list to populate the label:keyword dictionary

# from collections import Counter
# Counter(" ".join(df_view["content"]).split()).most_common(100) # slower than pd.Series()

# faster
# pd.Series(' '.join(df_view.content).lower().split()).value_counts()[:100]

## Creating Label:Keyword Dictionary

In [70]:
import spacy
from spacy.matcher import PhraseMatcher

# Creating keyword lists with string and split(', ').

keywords_list = ["""company, headquarter, corporate, finance, ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, inc., foundation, newspaper""".split(', '),
                 """university, students, bachelor, degree, school, academy, college, high, located, public, city, academy, county, institute, national, research, business, grammar, government, technology, medicine, mascot, academic, board, program, co-educational, economics, junior, science, schools, faculty""".split(', '),
                 """dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, english, american, chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, screenwriter, prose, poet, career, jazz, folk""".split(', '),
                 """rugby, player, football, professional, nfl, league, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, mlb, pitcher, fencer, driver, american, canadian, english, belgian, attended, season, champion, motorcycle""".split(', '),
                 """born, author, commentator, director, professor, leader, governor, politician, minister, president, died, representatives, assembly, republic, democratic, house of representatives, house, election, member of parliament, party, speaker, president, lawyer, liberal, candidate, election, deputy, prime minister, official""".split(', '),
                 """ship, aircraft, boeing, navy, destroyer, diesel, rail, warship, transport, submarine, trike, aerobatic, motorcar, railway, monoplane, vessel, motorcycle, navigation, railway, cars, airliner, naval, whaleship, rail, automobile, flown, ferry, wing, tailplane, car, convoy, flown, naval, locomotive, vehicle, flagship, cruise ship, boat, convoy, automaker, battleships""".split(', '),
                 """church, church, center, house, historic, dam, district, district, castle, hospital, institution, museum, victorian, farm, building, mall, restaurant, shopping mall, centre, supermarket, built in""".split(', '),
                 """river, mountain, hill, hills, land, lake, km, m, forest, creek, ocean, stream, strait, gulf, peak, elevation, glacier, volcanic, corona, tributary, mount, flows, border, watershed""".split(', '),
                 """rural, population, census, central, province, state, village, district, eastern, western, kilometres, km, mi, county, south, north, south-west, south-east, north-west, north-east, town, border, regional, capital, municipality, block, located, families, administrative, federation, india, united states, turkey, iran""".split(', '),
                 """habitat, species, beetles, subfamily, snail, endemic, family, extinct, shelled, squids, octopuses, wingspan, moth, frogs, dry land, humidity, fruit flies, genus, horse, racehorse, thoroughbred, farm, farms, frog, butterfly, class, flatworm, fish, bred, shark, tropical, subtropical, sphinx, dog, cat, mouse, lion, jaguar, sea, habitats, subclass, populations, fossil""".split(', '),
                 """plant, family, species, vegetative subspecies, dipterocarpaceae, tillandsia, genus, endemic, orchid, daisy, flower, flowering, plants, legume, habitat, green alga, lettuce, kelp, gutweed, succulent, microphylia, ulmus, coffee, soil, tropical, forest, wood, leaf, cultivated, tree, trees, aster, algae, sedge, grows, evergreen, fruit, seed, seeds, herbs, herb, bulrush, subtropical, violet, floral, meliaceae, wild, grass""".split(', '),
                 """album, country, singer, band, vocalist, member, guitarist, debut, studio, metal, records, produced, songs, live at, performance, indie, folk, musician, released, ep, music, rapper, official, cd, label, tracks, remastered, reissued, pop, release, recorded, producer, full-length, bonus, chart, reunited, grammy, billboard, featured, concert, singer-songwriter, songs""".split(', '),
                 """film, directed, starring, drama, based, released, novel, comedy, american, stars, produced, international, cinema, festival, documentary, biographical, love, romance, comedy-romance, movie""".split(', '),
                 """published, book, novel, journal, series, written, story, magazine, newspaper, daily, stories, peer-reviewed, fiction, covers, comic, volume, science, fantasy, edition, writer, law, created, research, established, history, weekly, issue, travel, academic, mystery, media, author, work, god""".split(', ')]

labels_list = ['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder',
               'MeanOfTransportation','Building', 'NaturalPlace', 'Village',
               'Animal', 'Plant', 'Album', 'Film', 'WrittenWork']

label_keyword_dict = dict(zip(labels_list, keywords_list))
print(label_keyword_dict['NaturalPlace'])

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

['river', 'mountain', 'hill', 'hills', 'land', 'lake', 'km', 'm', 'forest', 'creek', 'ocean', 'stream', 'strait', 'gulf', 'peak', 'elevation', 'glacier', 'volcanic', 'corona', 'tributary', 'mount', 'flows', 'border', 'watershed']


In [71]:
# Running nlp.make_doc to speed things up
for label in label_keyword_dict:
  patterns = [nlp.make_doc(text) for text in label_keyword_dict[label]]
  matcher.add(label, patterns)

In [72]:
doc = nlp("monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor.")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(rule_id, span.text)

WrittenWork writer
Artist writer
Artist actor


In [73]:
matches[0]

(10780820017166675169, 11, 12)

## Snorkel with Automated Labeling Functions

In [ ]:
# labeling function for all labels
@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x, label_keyword_dict):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    if labels: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

In [59]:
x = "monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor."

In [89]:
class all_labeling_functions(object):
    def __init__(self, labels):
        for label in labels:
            setattr(self, label, functools.partial(self._myfunction, label))
    
    @nlp_labeling_function() # labeling function for using spaCy
    def lf_labeler(x, label_keyword_dict=label_keyword_dict):
        labels = []
        doc = nlp(x) # tokenizing the input text
        matches = PhraseMatcher(doc) # finding all the token words that match a specific label
        
        for match_id in matches:
          labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
        
        if len(labels) != 0: # runs if labels is not empty
          return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
        
        else:
          return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

    def _myfunction(self, label, request):
        print(request)
        return "app/data/mydata_%s.csv" % (label)

lfs = []

for i in range(len(labels_list)):
  lf_labeler = all_labeling_functions(i)
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  lfs.append(all_labeling_functions(labels_list))

# myfunction = all_labeling_functions(labels_list)
# print(myfunction)

In [90]:
lfs

In [132]:
from types import FunctionType
from copy import copy

lf_list = []

@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    
    for match_id in matches:
      labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    
    if len(labels) != 0: # runs if labels is not empty
      return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    
    else:
      return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

  # namespace = sys._getframe(0).f_globals
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

def copy_function(fn, name):

    return FunctionType(name=name)

for label in labels_list:
    name = 'lf_labeler_' + str(label)
    lf_list.append(copy_function(lf_labeler, name))

lf_list

TypeError: ignored

In [112]:
from snorkel.labeling.lf.nlp import nlp_labeling_function
from functools import partial
import sys

namespace = sys._getframe(0).f_globals

def all_labeling_functions(label): # labeling function for all labels

  @nlp_labeling_function() # labeling function for using spaCy
  def lf_labeler(x):
      labels = []
      doc = nlp(x) # tokenizing the input text
      matches = PhraseMatcher(doc) # finding all the token words that match a specific label
      
      for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
      
      if len(labels) != 0: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
      
      else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

    # namespace = sys._getframe(0).f_globals
    # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  return lf_labeler

print('lol')

lfs = []

# lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
# for i in range(len(labels_list)):
#   lfs.append(lf_labeler_list[f'lf_labeler_{i}'])


# lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
for label in labels_list:
  lfs[f'lf_labeler_{label}'] = all_labeling_functions(label)

SyntaxError: ignored

In [110]:
lf_labeler_list

{'lf_labeler_0': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_1': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_2': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_3': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_4': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_5': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_6': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_7': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []]}

In [106]:
lfs

[functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=0),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=1),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=2),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=3),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=4),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=5),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=6),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=7)]

In [ ]:
# from snorkel.labeling import LabelingFunction


# def keyword_lookup(x, keywords, label):
#     if any(word in x.text.lower() for word in keywords):
#         return label
#     return ABSTAIN


# def make_keyword_lf(keywords, label=SPAM):
#     return LabelingFunction(
#         name=f"keyword_{keywords[0]}",
#         f=keyword_lookup,
#         resources=dict(keywords=keywords, label=label),
#     )

## Snorkel with Manual Labeling Functions

In [93]:
from snorkel.labeling.lf.nlp import nlp_labeling_function

@nlp_labeling_function()
def lf_company(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Company' for label_id in label_ids]):
        return Company # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_educational_institution(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'EducationalInstitution' for label_id in label_ids]):
        return EducationalInstitution # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_artist(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Artist' for label_id in label_ids]):
        return Artist # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_athlete(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Athlete' for label_id in label_ids]):
        return Athlete # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_office_holder(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'OfficeHolder' for label_id in label_ids]):
        return OfficeHolder # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_mean_of_transportation(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'MeanOfTransportation' for label_id in label_ids]):
        return MeanOfTransportation # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_building(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Building' for label_id in label_ids]):
        return Building # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_natural_place(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'NaturalPlace' for label_id in label_ids]):
        return NaturalPlace # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_village(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Village' for label_id in label_ids]):
        return Village # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_animal(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Animal' for label_id in label_ids]):
        return Animal # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_plant(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Plant' for label_id in label_ids]):
        return Plant # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_album(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Album' for label_id in label_ids]):
        return Album # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_film(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Film' for label_id in label_ids]):
        return Film # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_written_work(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'WrittenWork' for label_id in label_ids]):
        return WrittenWork # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

In [94]:
lfs = [
       lf_company,
       lf_educational_institution,
       lf_artist,
       lf_athlete,
       lf_office_holder,
       lf_mean_of_transportation,
       lf_building,
       lf_natural_place,
       lf_village,
       lf_animal,
       lf_plant,
       lf_album,
       lf_film,
       lf_written_work
]

## Applying Snorkel Labeling Functions

In [95]:
# company_dataset = dbpedia_train_df.iloc[0:1000]
# company_dataset

df_small_train_dbpedia = dbpedia_train_df.sample(1000, random_state=42).reset_index(drop=True)
df_small_test_dbpedia = dbpedia_test_df.sample(1000, random_state=42).reset_index(drop=True)
df_small_train_dbpedia

,labels,text
0,-1,sterling piano company the sterling piano company was a piano manufacturer in derby connecticut. the company was founded in 1873 by charles a. sterling as the sterling organ company. sterling had purchased the birmingham organ company in 1871 and had $30000 to fund the company. the sterling organ company began making pianos in 1885.
1,-1,nyc s-motor s-motor was the class designation given by the new york central to its alco-ge built s-1 s-2 s-2a and s-3 electric locomotives. the s-motors hold the distinction of being the world's first mass-produced main line electric locomotives with the prototype #6000 being constructed in 1904. the s-motors would serve alone until the more powerful t-motors began to arrive in 1913 eventually displacing them from main line passenger duties.
2,-1,axel zwingenberger axel zwingenberger (born may 7 1955 hamburg germany) is a blues and boogie-woogie pianist and songwriter. he is considered one of the finest boogie-woogie music masters in the world.
3,-1,sceptrophasma hispidulum sceptrophasma hispidulum commonly known as the andaman islands stick insect is a species of the stick insect family. it originates from the andaman islands and is commonly found in tropical forests there. they eat a variety of foliage though in captivity they commonly eat blackberry bramble hawthorn oak rose and lettuce. the species has the phasmid study group number psg183.
4,-1,nucet river (chiojdeanca) the nucet river is a tributary of the chiojdeanca river in romania.
...,...,...
995,-1,pat arrowsmith pat arrowsmith (born march 2 1930) is a british author and peace campaigner.arrowsmith was educated at cheltenham ladies college read history at the university of cambridge and then read social science at the university of liverpool and at ohio university as a us-uk fulbright scholar. she is a co-founder of the campaign for nuclear disarmament.she has served eleven prison sentences for her political activities.
996,-1,cleistanthus cleistanthus is a plant genus of the family phyllanthaceae. the genus comprises 140 species found from africa to the pacific islands. cleistanthus collinus is known for being toxic and frequently used for homicidal or suicidal purposes.
997,-1,kyunghyang shinmun the kyunghyang shinmun or kyonghyang sinmun is a major daily newspaper published in south korea. it is based in seoul. the name literally means urbi et orbi daily news.
998,-1,cyathea atropurpurea cyathea atropurpurea is a species of tree fern native to the islands of luzon mindanao leyte and mindanao in the philippines where it grows in forest at above 1000 m. the erect trunk is slender and may be up to 3 m tall. fronds are bipinnate and 1-2 m long. characteristically of this species the final pair of pinnae are usually reduced and occur towards the base of the stipe. these along with the stipe bases are persistent and retained around the trunk long after withering.


In [ ]:
from snorkel.labeling import PandasLFApplier
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_small_train_dbpedia)
L_test = applier.apply(df=df_small_test_dbpedia)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel







  0%|          | 0/1000 [00:00<?, ?it/s]






  0%|          | 2/1000 [00:01<12:57,  1.28it/s]






  0%|          | 3/1000 [00:03<17:55,  1.08s/it]






  0%|          | 4/1000 [00:04<20:23,  1.23s/it]






  0%|          | 5/1000 [00:06<22:36,  1.36s/it]






  1%|          | 6/1000 [00:08<23:15,  1.40s/it]






  1%|          | 7/1000 [00:09<24:56,  1.51s/it]






  1%|          | 8/1000 [00:11<26:26,  1.60s/it]






  1%|          | 9/1000 [00:13<26:44,  1.62s/it]






  1%|          | 10/1000 [00:14<26:19,  1.60s/it]






  1%|          | 11/1000 [00:16<26:46,  1.62s/it]






  1%|          | 12/1000 [00:18<27:23,  1.66s/it]






  1%|▏         | 13/1000 [00:20<27:50,  1.69s/it]






  1%|▏         | 14/1000 [00:21<28:38,  1.74s/it]






  2%|

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=14, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [ ]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

## Filtering out unlabeled data points

As we saw earlier, some of the data points in our train set received no labels from any of our LFs. These data points convey no supervision signal and tend to hurt performance, so we filter them out before training using a built-in utility.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)